# Lecture 4

# Tool Calling, function calling

# Note : This is one of the most concept that is as of today you can add business logic with just a python function and let llm use it.

Refer this https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

In [1]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: uvicorn 0.14.0 has a non-standard dependency specifier click>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of uvicorn or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")



# Understanding Tool Calling



Import Libraries

In [65]:

from langchain_core.messages import HumanMessage,ToolMessage
from langchain_core.tools import tool


Define Tool


In [66]:
@tool
def bmi_calculator(weight: float, height: float) -> str:
    """
    Calculate the BMI of a person given their weight and height.
    """
    bmi=weight/(height**2)
    if bmi < 18.5:
        return "Underweight"
    elif bmi >= 18.5 and bmi < 24.9:
        return "Normal weight"
    elif bmi >= 24.9 and bmi < 29.9:
        return "Overweight"
    else:
        return "Obesity"

tools = [bmi_calculator]


Bind your llm with tools


In [67]:
llm_with_tool = llm.bind_tools(tools)


In [76]:
query = "What is the BMI of a person who is 100kg and 1.75m tall?"


In [77]:
message = [HumanMessage(content=query)]

ai_message = llm_with_tool.invoke(message)

message



[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall?', additional_kwargs={}, response_metadata={})]

In [78]:
ai_message


AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-41480096-b34c-469c-8720-9bdc3a6abb08-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bee2adbd-f987-44e3-9d36-cf837e63fedb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 79, 'output_tokens': 5, 'total_tokens': 84, 'input_token_details': {'cache_read': 0}})

In [79]:
message.append(ai_message)

print(message)

[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-41480096-b34c-469c-8720-9bdc3a6abb08-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bee2adbd-f987-44e3-9d36-cf837e63fedb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 79, 'output_tokens': 5, 'total_tokens': 84, 'input_token_details': {'cache_read': 0}})]


In [80]:
for i in ai_message.tool_calls:
    print(i)



{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bee2adbd-f987-44e3-9d36-cf837e63fedb', 'type': 'tool_call'}


In [81]:
for i in ai_message.tool_calls:
    selected_tool = {"bmi_calculator": bmi_calculator}[i['name'].lower()]
    print(selected_tool)




name='bmi_calculator' description='Calculate the BMI of a person given their weight and height.' args_schema=<class 'langchain_core.utils.pydantic.bmi_calculator'> func=<function bmi_calculator at 0x000002401AB5E8B0>


In [82]:
for i in ai_message.tool_calls:
    selected_tool = {"bmi_calculator": bmi_calculator}[i['name'].lower()]
    print(selected_tool)
    tool_output = selected_tool.invoke(i['args'])
    print(tool_output)
    message.append(ToolMessage(content=tool_output, tool_call_id=i['id']))


print(message)




name='bmi_calculator' description='Calculate the BMI of a person given their weight and height.' args_schema=<class 'langchain_core.utils.pydantic.bmi_calculator'> func=<function bmi_calculator at 0x000002401AB5E8B0>
Obesity
[HumanMessage(content='What is the BMI of a person who is 100kg and 1.75m tall?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'bmi_calculator', 'arguments': '{"height": 1.75, "weight": 100.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-41480096-b34c-469c-8720-9bdc3a6abb08-0', tool_calls=[{'name': 'bmi_calculator', 'args': {'height': 1.75, 'weight': 100.0}, 'id': 'bee2adbd-f987-44e3-9d36-cf837e63fedb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 79, 'output_tokens': 5, 'total_tokens': 84, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Obesity', tool_call_id='bee2adbd-f987-44e

In [83]:
llm_with_tool.invoke(message).content


'The BMI is in the Obesity range.'